<a href="https://colab.research.google.com/github/pariyamd/CPATH_TTA/blob/main/Project_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers, optimizers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/COMP499/Project/Training data

/content/drive/MyDrive/COMP499/Project/Training data


In [5]:
!tar -xf 02_training_native.tar -C "/content/drive/MyDrive/COMP499/Project/Training data"

In [6]:
# Import ResNet50 architecture with ImageNet weights
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(350,350,3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Make the convolutional base trainable
conv_base.trainable = True
set_trainable = False

# Adjust layer names based on the ResNet50 architecture
for layer in conv_base.layers:
    if layer.name == 'conv5_block1_2_conv' or layer.name == 'conv5_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-5),
              metrics=['acc'])

94765736/94765736 [==============================] - 3s 0us/step


In [7]:
#folder with training dataset
train_dir = '/content/drive/MyDrive/COMP499/Project/Training data/02_training_native'

# Data generator
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

#Augmentation: horizontal and vertical flips
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip=True,
        vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(350,350),
        batch_size=100,
        class_mode='categorical')

#train for 7 Epochs
history = model.fit(train_generator, epochs=7)

Found 120000 images belonging to 3 classes.
Epoch 1/7
1200/1200 [==============================] - 1706s 1s/step - loss: 0.6523 - acc: 0.7146
Epoch 2/7
1200/1200 [==============================] - 1066s 888ms/step - loss: 0.5417 - acc: 0.7671
Epoch 3/7
1200/1200 [==============================] - 1084s 903ms/step - loss: 0.5013 - acc: 0.7868
Epoch 4/7
1200/1200 [==============================] - 1059s 882ms/step - loss: 0.4720 - acc: 0.8013
Epoch 5/7
1200/1200 [==============================] - 1045s 871ms/step - loss: 0.4515 - acc: 0.8110
Epoch 6/7
1200/1200 [==============================] - 1074s 895ms/step - loss: 0.4321 - acc: 0.8209
Epoch 7/7
1200/1200 [==============================] - 1064s 886ms/step - loss: 0.4130 - acc: 0.8294


In [9]:
# Save model and weights
model.save_weights('/content/drive/MyDrive/COMP499/Project/TvN_350_SN_D256_Initial_Ep7.weights')
model.save('/content/drive/MyDrive/COMP499/Project/TvN_350_SN_D256_Initial_Ep7.keras')

In [12]:
from tensorflow.compat.v1 import Session, ConfigProto, GPUOptions
from tensorflow.compat.v1.keras.backend import set_session

# Clear TensorFlow session to free up GPU memory
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
config.gpu_options.allow_growth = True
set_session(Session(config=config))

# After clearing, reset Keras session
from tensorflow.keras import backend as K
K.clear_session()

In [13]:
import torch

# Clear PyTorch's CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## TRAIN IN PYTORCH

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import Adam
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the ResNet50 model pre-trained on ImageNet
conv_base = models.resnet50(weights='DEFAULT')

# Since we're using include_top=False in Keras, we'll replace the FC layers ourselves
num_ftrs = conv_base.fc.in_features  # Get number of features in last layer
conv_base.fc = nn.Sequential(  # Replace the fully connected layer
    nn.Flatten(),
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, 3),
    nn.Softmax(dim=1)
)

conv_base = conv_base.to(device)  # Move model to GPU if available

# Freeze all layers except 'layer4[0].conv1'
# Make the convolutional base trainable and unfreeze layers after 'layer4[0].conv1'
for name, child in conv_base.named_children():
    if name in ['layer4']:
        for param in child.parameters():
            param.requires_grad = True
    else:
        for param in child.parameters():
            param.requires_grad = False

# Data augmentation and normalization for training
# Just normalization for validation
train_transforms = transforms.Compose([
    transforms.Resize((350, 350)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Assuming '/content/drive/MyDrive/COMP499/Project/Training data/02_training_native' is your training directory
train_dataset = ImageFolder(root='/content/drive/MyDrive/COMP499/Project/Training data/02_training_native', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(conv_base.parameters(), lr=1e-5)

# Training loop
num_epochs = 7
for epoch in range(num_epochs):
    conv_base.train()  # Set the model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}')

    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the configured device

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = conv_base(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct_preds / total_preds

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')


Epoch 1/7: 100%|██████████| 1200/1200 [36:01<00:00,  1.80s/it]


Epoch [1/7], Loss: 0.8583, Accuracy: 79.92%


Epoch 2/7: 100%|██████████| 1200/1200 [28:24<00:00,  1.42s/it]


Epoch [2/7], Loss: 0.6809, Accuracy: 91.65%


Epoch 3/7: 100%|██████████| 1200/1200 [28:25<00:00,  1.42s/it]


...

Epoch [5/7], Loss: 0.6139, Accuracy: 95.85%


Epoch 6/7: 100%|██████████| 1200/1200 [27:51<00:00,  1.39s/it]


Epoch [6/7], Loss: 0.6063, Accuracy: 96.29%


Epoch 7/7: 100%|██████████| 1200/1200 [27:45<00:00,  1.39s/it]

Epoch [7/7], Loss: 0.6010, Accuracy: 96.67%


In [4]:
torch.save(conv_base, '/content/drive/MyDrive/COMP499/Project/TvN_350_SN_D256_Initial_Ep7_fullmodel.pth')

In [5]:
# Clear PyTorch's CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [7]:
##Consecutive stepwise release of further convolutional layers
# v2, Layers 3, 4 free
conv_base.train()

for name, child in conv_base.named_children():
    if name == 'layer3':
        for param in child.parameters():
            param.requires_grad = True

# Update the optimizer to include the newly unfrozen parameters
optimizer = Adam(filter(lambda p: p.requires_grad, conv_base.parameters()), lr=1e-6)

# Continue training or fine-tuning
for epoch in range(1):  # Fine-tune for 1 epoch
    conv_base.train()  # Ensure the model is in training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}')

    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the configured devic
        optimizer.zero_grad()

        outputs = conv_base(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()

    # Compute loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct_preds / total_preds
    print(f'Fine-tune Epoch, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')


Epoch 1/7: 100%|██████████| 1200/1200 [30:38<00:00,  1.53s/it]

Fine-tune Epoch, Loss: 0.5968, Accuracy: 96.97%


In [8]:
torch.save(conv_base, '/content/drive/MyDrive/COMP499/Project/Models/TvN_350_SN_D256_v2_Ep1_fullmodel.pth')